In [ ]:
%matplotlib inline

# Calculating the initial piece of coastal and along-coast connectivity

There are multiple analysis steps to get to various perspectives on coastal connectivity, but for all of them, this is the first step.

## Making coastal paths

In projects/shelf_transport, I have worked on the script make_coastal_paths.py, which reads in a path along the active rho points that are along the coast but not including the bays, then finds points along the coast every 5km and 5km across coast from there, making boxes. See the figures below for an idea of this. The boxes are in orange. The idea is that I will look at the connectivity from and to each of the boxes and then make a matrix showing the connections. This works since it will be along-coast, whereas looking every where in map space is too hard to put in a matrix. Note that I will need to normalize by area at the end since the boxes aren’t all the same size.

![coast boxes1](images/coastboxes1.png)

![coast boxes2](images/coastboxes2.png)


From Evernote note ["Creating along-coast path boxes"](https://www.evernote.com/shard/s367/nl/2147483647/c7cf7213-a724-402a-ad6e-47fb8fd9dd51/).

Made with shelf_transport/make_conn_plots.plot_domain() to show the outline of the boxes (all together) and a feel for the numbering of the boxes along the coast, up to 342 boxes.

![coast boxes 3](images/coastboxes3.png)

From Evernote note ["Map or key of along-coast boxes"](https://www.evernote.com/shard/s367/nl/2147483647/6d17fb7a-cc66-4739-85ab-6d8dfe745d59/).

In [ ]:
# %load ../make_coastal_paths.py
'''
Find paths defining 3km across- and along-shelf boxes for examining
coastal connectivity.
'''

import numpy as np
import tracpy
import tracpy.plotting
from matplotlib.path import Path
import matplotlib.patches as patches
import matplotlib.pyplot as plt

loc = 'http://barataria.tamu.edu:8080/thredds/dodsC/NcML/txla_nesting6.nc'
# grid_filename = '/atch/raid1/zhangxq/Projects/txla_nesting6/txla_grd_v4_new.nc'
proj = tracpy.tools.make_proj('nwgom')
grid = tracpy.inout.readgrid(loc, proj)

# To click to get the points:
# tracpy.plotting.background(grid)
# plt.plot(grid['xpsi'], grid['ypsi'], 'k', grid['xpsi'].T, grid['ypsi'].T, 'k')
# ind = grid['mask'].astype(bool)
# plt.plot(grid['xr'][ind], grid['yr'][ind], 'bs')
# pts = ginput(timeout=0, n=0)
# np.savez('calcs/coastpts.npz', lon=lon, lat=lat)

# Read in previously-clicked rho points defining the inner edge of
# the inner shelf (excludes the bays)
d = np.load('calcs/coastpts.npz')
lon = d['lon']; lat = d['lat']
d.close()
x, y = proj(lon, lat)

## Add in more points
# Calculate distance along-shore for each line
dd = np.sqrt((x[1:]-x[:-1])**2 + (y[1:]-y[:-1])**2)
d = np.zeros(x.size)
d[1:] = np.cumsum(dd)
while dd.max()>100:
    xnew = np.empty(x.size*2-1)
    xnew[1::2] = .5*(x[1:]+x[:-1])
    xnew[::2] = x
    ynew = np.empty(y.size*2-1)
    ynew[1::2] = .5*(y[1:]+y[:-1])
    ynew[::2] = y
    x = xnew.copy(); y = ynew.copy()
    dd = np.sqrt((x[1:]-x[:-1])**2 + (y[1:]-y[:-1])**2)
    d = np.zeros(x.size)
    d[1:] = np.cumsum(dd)

ds = 5 # spacing at ds km along the coast
d3 = np.arange(d.min(), d.max(), 1000*ds)
x3 = np.interp(d3, d, x)
y3 = np.interp(d3, d, y)

# Calculate normal vector
dx = x3[1:] - x3[:-1]
dy = y3[1:] - y3[:-1]
mag = np.sqrt(dx**2 + dy**2)
# subtract one since also backing up the x3, y3 line in the following chunk of code
xn = x3[:-1] + (dy/mag)*1000*(ds-1)
yn = y3[:-1] - (dx/mag)*1000*(ds-1)

# Shift the original line back onto land to capture all active space
x3 = x3[:-1] - (dy/mag)*1000
y3 = y3[:-1] + (dx/mag)*1000

lonn, latn = grid['basemap'](xn, yn, inverse=True)
lon3, lat3 = grid['basemap'](x3, y3, inverse=True)
xg3, yg3, _ = tracpy.tools.interpolate2d(x3, y3, grid, 'd_xy2ij')
xgn, ygn, _ = tracpy.tools.interpolate2d(xn, yn, grid, 'd_xy2ij')

# put the 4 parts together to make path for each box along-shore
paths = []; pathsxy = []; pathsg = []
for i in xrange(xn.size-1):
    vertsg = [(xg3[i+1],yg3[i+1]), (xgn[i+1],ygn[i+1]), 
            (xgn[i],ygn[i]), (xg3[i],yg3[i])]
    vertsxy = [(x3[i+1],y3[i+1]), (xn[i+1],yn[i+1]), 
            (xn[i],yn[i]), (x3[i],y3[i])]
    verts = [(lon3[i+1],lat3[i+1]), (lonn[i+1],latn[i+1]), 
            (lonn[i],latn[i]), (lon3[i],lat3[i])]
    pathsg.append(Path(vertsg))
    pathsxy.append(Path(vertsxy))
    paths.append(Path(verts))

# make outer path of all boxes
verts_outerg = np.vstack((np.vstack((xg3,yg3)).T, 
                        np.vstack((xgn[::-1],ygn[::-1])).T,
                        [xg3[0],yg3[0]]))
outerpathg = Path(verts_outerg)
verts_outerxy = np.vstack((np.vstack((x3,y3)).T, 
                        np.vstack((xn[::-1],yn[::-1])).T,
                        [x3[0],y3[0]]))
outerpathxy = Path(verts_outerxy)

np.savez('calcs/coastpaths.npz', paths=paths, pathsg=pathsg, 
            pathsxy=pathsxy, outerpathg=outerpathg, outerpathxy=outerpathxy)

# Plot up
fig = plt.figure()
ax = fig.add_subplot(111)
tracpy.plotting.background(grid, ax=ax)
plt.plot(grid['xpsi'], grid['ypsi'], 'k', grid['xpsi'].T, grid['ypsi'].T, 'k')
ind = grid['mask'].astype(bool)
plt.plot(grid['xr'][ind], grid['yr'][ind], 'bs')
for path in pathsxy:
    patch = patches.PathPatch(path, facecolor='orange', lw=2, zorder=10)
    ax.add_patch(patch)
plt.plot(verts_outerxy[:,0], verts_outerxy[:,1], 'r-', lw=5, zorder=15)

## Running analysis on coastal paths

Using the coastal boxes calculated, I then find when drifters move between the boxes. This is done in `shelf_transport/find_coastal_path_connectivity.py` and run on hafen. The basic function of this is to first find drifters that are in any coastal boxes ever in the simulation to only further include those drifters in analysis (these drifter indices are saved in `iinside`). Then, up to 5 entrance times of any of those drifters into each coastal boxes are saved in `inbox` (shape is number of boxes x number of drifters in any coastal box for this simulation x 5 possible crossings), and the same for exit times in `outbox`.

This analysis produces files of the type `shelf_transport/calcs/alongcoastconn/2004-01-01.npz`, run and saved on hafen.

In [ ]:
# %load ../find_coastal_path_connectivity.py
'''
Script to do analysis for what drifters cross the shelf. This will be run on hafen,
where all of the tracks files are.

Run from shelf_transport directory.
'''

import matplotlib as mpl
mpl.use("Agg") # set matplotlib to use the backend that does not require a windowing system
import numpy as np
import os
import netCDF4 as netCDF
import pdb
import matplotlib.pyplot as plt
import tracpy
import init
from datetime import datetime, timedelta
import glob
from matplotlib.mlab import find, Path
import time

def load_tracks(File):
    '''
    Load in and return tracks from File
    '''

    # Get necessary info from File
    d = netCDF.Dataset(File)
    xg = d.variables['xg'][:]
    yg = d.variables['yg'][:]
    #tp = d.variables['tp'][:]
    # Calculate times since they are messed up if a drifter exits the domain
    tseas = d.variables['tseas'][:]
    N = d.variables['N'][:]
    iday = int((3600.*24)/(tseas/N))
    days = np.linspace(0, 30, xg.shape[1]) #tseas/N/(3600.*24)) # forward or backward, but doesn't matter here I guess
    # pdb.set_trace()
    d.close()

    return xg, yg, iday, days



def run():

    # Find files to run through
    # Files = glob.glob('tracks/2014-0[1,2]-*gc.nc')
    # Files.extend(glob.glob('tracks/2014-0[7,8]-*gc.nc'))
    Files = glob.glob('tracks/2008-??-*gc.nc')

    # grid_filename = '/atch/raid1/zhangxq/Projects/txla_nesting6/txla_grd_v4_new.nc'
    # vert_filename='/atch/raid1/zhangxq/Projects/txla_nesting6/ocean_his_0001.nc'
    # grid = tracpy.inout.readgrid(grid_filename, vert_filename=vert_filename, usebasemap=True)

    # load in paths for coastline boxes
    d = np.load('calcs/coastpaths.npz') # use paths in grid space
    paths = d['pathsg']
    pathouter = d['outerpathg'].item()
    d.close()

    # Loop through files for analysis
    for File in Files:

        savefile = 'calcs/alongcoastconn/' + File.split('/')[-1][:-5] + '.npz'
        if os.path.exists(savefile): # don't repeat calc if last file was created
            continue

        # import pdb; pdb.set_trace()
        # print(File)
        # import time
        # stime = time.time()
        xg, yg, iday, days = load_tracks(File)

        # Change to projected drifter locations now
        nanind = (xg==-1) # indices where nans are location in xg, yg; for reinstitution of nans
        xg[nanind] = np.nan; yg[nanind] = np.nan

        # Remove points that never enter the outer box paths
        inouter = pathouter.contains_points(np.vstack((xg.flat, yg.flat)).T).reshape(xg.shape)
        # print('made it past contains_points')
        # print(time.time()-stime)
        iinside = find(inouter.sum(axis=1).astype(bool)) # indices of drifters that go inside
        # print '0 time for outer path comparison: ', time.time() - tic_start
        # save a pointer of these drifters within the large arrays of drifters
        xgin = xg[iinside,:]; ygin = yg[iinside,:]

        # Initial array of information
        ncrosses = 5
        inbox = np.ones((len(paths), iinside.size, ncrosses))*np.nan # to store analysis.
        outbox = np.ones((len(paths), iinside.size, ncrosses))*np.nan # to store analysis.

        for i,path in enumerate(paths):

            # which points are inside the regions
            inside = path.contains_points(np.vstack((xgin.flat, ygin.flat)).T).reshape(xgin.shape)
            whencross = np.diff(inside.astype(int), axis=1) # will pick out when the drifters change from outside to inside region or vice versa
            whencross_sorted = np.sort(whencross, axis=1) # will pick out when the drifters change from outside to inside region or vice versa
            isort = np.argsort(whencross, axis=1)
            outbox[i,:,:] = days[isort[:,:ncrosses]] # allow for up to ncrosses re-exits
            # nan out the entries that aren't actually exiting
            iout = whencross_sorted[:,:ncrosses]!=-1
            outbox[i,iout] = np.nan
            outbox[i,:,:] = np.sort(outbox[i,:,:], axis=1)

            inbox[i,:,:] = days[isort[:,-ncrosses:]] # allow for up to ncrosses re-enters
            # nan out the exits that aren't actually exiting
            iin = whencross_sorted[:,-ncrosses:]!=1
            inbox[i,iin] = np.nan
            inbox[i,:,:] = np.sort(inbox[i,:,:], axis=1)

        # save array
        np.savez(savefile, inbox=inbox, outbox=outbox, iinside=iinside)



if __name__ == "__main__":
    run()


## Where this is being used

Subsequent to the previously calculations, the analysis of drifters reaching coastal boxes are used in:

* coastal connectivity and coastal vulnerability work (MPB paper) with analysis on connections from the rest of the numerical domain to the coast boxes, and examining the numbers of drifters reaching each coastal box in time:
 * connectivity to the coast is summed in `shelf_transport/make_conn_plots.run()`. This makes files of the type: `calcs/alongcoastconn/conn-2004-01.npz`
 * coastal connectivity plots are made in `shelf_transport/plot_conn2coast_2boxes.py` or `shelf_transport/plot_conn2coast.py`
 * coastal vulnerability plots are made in `shelf_transport/plot_vulnerability_insteps.py` and `shelf_transport/plot_vulnerability.py`.
 * an example to illustrate drifters being counted if they reach the coastal area is created in `shelf_transport/plot_sampledrifters.py`.

* [along-coast connectivity work](Along-coast connectivity.ipynb): likelihood of drifters exiting a given box and entering another, in time. This is examined particularly with respect to being near bay entrances
 * along-coast connectivity is set up by summing files calculated in the previous section in `shelf_transport/make_conn_plots.run_in_time()`. This makes files of the type: `calcs/alongcoastconn/conn_in_time/2004-01-01T00.npz`.
 * plots are made in `shelf_transport/make_conn_plots.py`.